In [62]:
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [63]:
with open('rf.pkl', 'rb') as file:
    rf = pickle.load(file)

# Load the GBM model
with open('gbm_model.pkl', 'rb') as file:
    gbm = pickle.load(file)

# Load genre_encoder
with open('genre_encoder.pkl', 'rb') as file:
    genre_encoder = pickle.load(file)

# Load artist_encoder
with open('artist_encoder.pkl', 'rb') as file:
    artist_encoder = pickle.load(file)


In [65]:
def calculate_distance(pred, actual):
    # Calculate distance between predicted and actual values
    return abs(pred - actual)

In [66]:
songs_data = pd.read_csv('data/spotify_data_test_with_genre.csv')

In [67]:
distances_between_distances = []
test_list = songs_data.drop('popularity', axis=1)
test_pop = songs_data['popularity']

genre_dummies = pd.get_dummies(test_list['genre'])
artist_dummies = pd.get_dummies(test_list['artists'])

# Drop the 'lyrics' column from X
# X = X.drop('lyrics', axis=1)
test_list = test_list.drop('genre', axis=1)
test_list = test_list.drop('artists', axis=1)
test_list = test_list.drop('names', axis=1)
#X = X.drop('releaseMonth', axis=1)

# Concatenate the one-hot encoded lyrics with the remaining features in X
# X = pd.concat([X, lyrics_dummies], axis=1)
test_list = pd.concat([test_list, artist_dummies], axis=1)
test_list = pd.concat([test_list, genre_dummies], axis=1)

song_list = songs_data['names']

In [68]:
print(test_list['timeSinceRelease'])

0     169
1     163
2     177
3     169
4     211
     ... 
85    152
86    253
87    352
88    281
89    491
Name: timeSinceRelease, Length: 90, dtype: int64


In [32]:
print(song_list)

0                Creepin' (with The Weeknd & 21 Savage)
1                                             Kill Bill
2                                                   AMG
3     Superhero (Heroes & Villains) [with Future & C...
4                                             Anti-Hero
                            ...                        
85                                                Ditto
86                                            Thank God
87                                        La Curiosidad
88                                  Next Thing You Know
89                                             snowfall
Name: names, Length: 90, dtype: object


In [82]:
GBMDIF = []
RFDIF = []
i = 0
for name in song_list:
    # index = song_list.index(name)
    found = songs_data.loc[i]
    givenSong = pd.DataFrame({'length': found['length'], 'tempo': found['tempo'], 'timeSinceRelease': found['timeSinceRelease'], 'releaseMonth': found['releaseMonth'], 'artists': found['artists'], 'genre':found['genre']}, index=[0])
    actual_pop = test_pop[i]
    print(givenSong)
    #genre_dummies = pd.get_dummies(givenSong['genre'])
    #artist_dummies = pd.get_dummies(givenSong['artists'])

# Drop the 'lyrics' column from X
# X = X.drop('lyrics', axis=1)
    #givenSong = givenSong.drop('genre', axis=1)
    #givenSong = givenSong.drop('artists', axis=1)
    #givenSong = givenSong.drop('names', axis=1)
    #givenSong = givenSong.drop('popularity', axis=1)
#X = X.drop('releaseMonth', axis=1)

# Concatenate the one-hot encoded lyrics with the remaining features in X
# X = pd.concat([X, lyrics_dummies], axis=1)
    #givenSong = pd.concat([givenSong, artist_dummies], axis=1)
    #givenSong = pd.concat([givenSong, genre_dummies], axis=1)

    genre_encoder = LabelEncoder()
    artist_encoder = LabelEncoder()

# = pd.get_dummies(X['genre'])
#artist_dummies = pd.get_dummies(X['artists'])

# Drop the 'lyrics' column from X
# X = X.drop('lyrics', axis=1)
#X = X.drop('genre', axis=1)
#X = X.drop('artists', axis=1)

#X = X.drop('releaseMonth', axis=1)

    givenSong['genre'] = genre_encoder.fit_transform(givenSong['genre'])
    givenSong['artists'] = artist_encoder.fit_transform(givenSong['artists'])

    #print(givenSong['popularity'])

    #givenSong = givenSong.drop('names', axis=1)
    print('the gbm diff')
    print(actual_pop - gbm.predict(givenSong))
    print('the rf diff')
    print(actual_pop - rf.predict(givenSong))
    GBMDIF.append(actual_pop - gbm.predict(givenSong))
    RFDIF.append(actual_pop - rf.predict(givenSong))
    i=i+1

   length  tempo  timeSinceRelease  releaseMonth       artists genre
0  221520  97.95               169            12  Metro Boomin   rnb
the gbm diff
[15.90133091]
the rf diff
[16.34]
   length  tempo  timeSinceRelease  releaseMonth artists genre
0  153946  88.98               163            12     SZA   pop
the gbm diff
[21.20226038]
the rf diff
[18.51]
   length    tempo  timeSinceRelease  releaseMonth        artists   
0  174942  136.175               177            11  Natanael Cano  \

               genre  
0  Corridos tumbados  
the gbm diff
[23.80378893]
the rf diff
[18.41]
   length    tempo  timeSinceRelease  releaseMonth       artists genre
0  182666  116.622               169            12  Metro Boomin   rap
the gbm diff
[16.17027711]
the rf diff
[13.11]
   length   tempo  timeSinceRelease  releaseMonth       artists genre
0  200690  97.008               211            10  Taylor Swift   pop
the gbm diff
[23.53281222]
the rf diff
[16.4]
   length   tempo  timeSinceRelease

In [83]:
differences = []

for i in range(len(GBMDIF)):
    diff = GBMDIF[i] - RFDIF[i]
    differences.append(diff)


In [84]:
print(differences)

[array([-0.43866909]), array([2.69226038]), array([5.39378893]), array([3.06027711]), array([7.13281222]), array([2.41067741]), array([4.48829286]), array([-1.63600827]), array([4.68550233]), array([2.00378529]), array([1.4822757]), array([4.28847438]), array([3.62027711]), array([3.00180585]), array([1.55144256]), array([2.69226038]), array([2.4715296]), array([5.80644603]), array([4.20336927]), array([5.09444408]), array([3.67752126]), array([3.87154644]), array([3.84354148]), array([1.23233064]), array([5.73544563]), array([7.27442572]), array([3.90496975]), array([1.98179521]), array([1.18306303]), array([4.37260111]), array([5.46932617]), array([5.61048568]), array([5.41444408]), array([4.34516788]), array([3.50258511]), array([1.40730396]), array([5.28950169]), array([3.75354148]), array([5.08950169]), array([4.14550233]), array([5.85927901]), array([1.50297928]), array([6.21048568]), array([5.79911482]), array([2.38729337]), array([2.16829577]), array([2.67226038]), array([0.737